In [10]:

# Get party information and Twitter UIDs for politicians
# Both CSVs were downloaded from http://twitterpoliticians.org/download

import pandas as pd
import tweepy as tw
import numpy as np
from os import environ as env
import streamlit as st
from transformers import pipeline

In [ ]:
st.title('Tweets - Republican or Democrat?')

In [5]:
# Load credentials from .env file
auth = tw.AppAuthHandler(env["CONSUMER_KEY"], env["CONSUMER_SECRET"])

# Authorize
api = tw.API(auth)

In [6]:
# Get data
SCREEN_NAMES = [
    "SpeakerPelosi",
    "RepMaxineWaters", 
    "SenTedCruz", 
    "RandPaul",
    "GOPLeader", 
    "Jim_Jordan", 
    "RepAdamSchiff", 
    "ewarren"  
]

DEMOCRATS = [
    "SpeakerPelosi",
    "RepMaxineWaters",
    "RepAdamSchiff", 
    "ewarren"
]

MAX_TWEETS = 10

dfs_list = []
for name in SCREEN_NAMES:
    try:
        # Creation of query method using parameters
        tweets = tw.Cursor(api.user_timeline, screen_name=name, tweet_mode="extended").items(MAX_TWEETS)

        # Post-process tweets
        tweets_json = [t._json for t in tweets]
        tweets_normalized = pd.json_normalize(tweets_json)
        dfs_list.append(tweets_normalized)
    except:
        print('Unable to find tweets for {}'.format(name))
      
# Combine all dfs into one df and clean
tweets_df = pd.concat(dfs_list)
tweets_df = tweets_df[["created_at", "id", "full_text", "user.id_str", "user.name", "user.screen_name"]]

# tweets_df.rename({"id": "tweet_id",
#     "user.id_str": "uid",
#     "user.name": "username", 
#     "user.screen_name": "screen_name",
#     "user.url": "user_url"
# }, inplace=True)

# Add column for political party
tweets_df['party'] = np.where(tweets_df['user.screen_name'].isin(DEMOCRATS) , 'democrat', 'republican')

st.subheader('Raw tweet data')
st.write(tweets_df)

In [11]:
# Classify tweets
classifier = pipeline("zero-shot-classification")

No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)


RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.